ImportError: No module named tensorbard.plugins

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data  
from tensorflow.contrib.tensorboard.plugins import projector

mnist = input_data.read_data_sets('MNIST_data',one_hot=True)
# 运行次数
max_steps = 1001
## 图片数量
image_num = 3000
#文件路进
DIR ='/var/opt/pycharm/tensorflow-10-classes/'
# 定义绘会话
sess = tf.Session()

#载入图片
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name='embedding')


##参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) # 平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histgram',var) # 直方图

## 命名空间
with tf.name_scope("input"):
    x=tf.placeholder(tf.float32,[None,784],name="x-input")
    y = tf.placeholder(tf.float32,[None,10],name="y-input")


# 显示图皮那
with tf.name_scope("input_reshape"):
    # -1 表示任意值,原来一张图片是一行1x784,现在百纳成28x28,channel=1因为是黑白图片
    image_shaped_input = tf.reshape(x,[-1,28,28,1])
    tf.summary.image("input",image_shaped_input,10)

with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        W = tf.Variable(tf.zeros([784,10]))
        variable_summaries(W) ## 分析w
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]))
        variable_summaries(b) # 分析b
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,W)+b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)
    
    
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)


sess.run(tf.global_variables_initializer())

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1)) ## argmax 返回一维张量中最大的值所在的位置
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
        
## 生成metadata 文件
if tf.gfile.Exists(DIR+'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR+'projector/projector/metadata.tsv')
with open(DIR+'projector/projector/metadata.tsv','w+') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:],1))
    for  i in range(image_num):
        f.write(str(labels[i])+'\n')


merged = tf.summary.merge_all()

projector_writer = tf.summary.FileWriter(DIR+'projector/projector',sess.graph)
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = DIR+'projector/projector/metadata.tsv'
embed.sprite.image_path = DIR +'projector/data/mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer,config)

for i in range(max_steps):
    ## 每个批次 100个 样本
    batch_xs,batch_ys = mnist.train.next_batch(100)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary,_=sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata,'step%03d'%i)
    projector_writer.add_summary(summary,i)
    
    if i%100 == 0:  
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print ("Iter" + str(i)+" ,Testing Accuracy" +str(acc))

saver.save(sess,DIR+'projector/projector/a_model.ckpt',global_step=max_steps)
projector_writer.close()
sess.close()

    
        

Extracting MNIST_data/train-images-idx3-ubyte.gz


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Iter0 ,Testing Accuracy0.1013


Iter100 ,Testing Accuracy0.7122


Iter200 ,Testing Accuracy0.8004


...

Iter600 ,Testing Accuracy0.8256


Iter700 ,Testing Accuracy0.8417


Iter800 ,Testing Accuracy0.8651


Iter900 ,Testing Accuracy0.8803


Iter1000 ,Testing Accuracy0.8891
